In [ ]:
#!pip3 install --upgrade pip
#!pip3 uninstall -y numpy
#!pip3 install --user --upgrade --force-reinstall tensorflow-gpu==1.1.0 
!pip install --user --upgrade wrapt
!pip install tensorrec 
!pip install fuzzywuzzy
!pip install python-Levenshtein
#!pip3 install --upgrade numpy

Requirement already up-to-date: wrapt in c:\users\612987997\appdata\local\programs\python\python37\lib\site-packages (1.11.2)


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from collections import defaultdict
import csv

import random
from scipy import sparse
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

import numpy as np
import boto3
import gzip
import pandas as pd
from io import BytesIO, TextIOWrapper
import tensorrec
import io
import os
from datetime import date
import math
from fuzzywuzzy import  fuzz

import logging
logging.getLogger().setLevel(logging.INFO)


In [32]:
# To change to Athena when we have the data in there -- to be replaced with non-static when pipeline is implemented 
bucket = "bt-data-science-playground"
file_name = "GP/device_hist3.csv.gz"
 
s3 = boto3.client('s3') 
# 's3' is a key word. create connection to S3 using default config and all buckets within S3

obj = s3.get_object(Bucket= bucket, Key= file_name) 
# get object and file (key) from bucket

dataset = pd.read_csv(gzip.open(obj['Body']),encoding = "ISO-8859-1",
                      dtype = {'REFERENCE_DATE':'str',
                               'SUB_ID':'int',
                               'MARKETING_NAME':'str',
                               'INTERACTIONS':'int'
                                }) # 'Body' is a key word

dataset['MARKETING_NAME'] = dataset['MARKETING_NAME'].map(lambda x: x.rstrip())
dataset.head()

,Unnamed: 0,REFERENCE_DATE,SUB_ID,MARKETING_NAME,INTERACTIONS
0,0,NOT IMPORTANT,1006,SAMSUNG GALAXY NOTE 8 LTE,1
1,1,NOT IMPORTANT,1016,SONY XPERIA XA LTE,1
2,2,NOT IMPORTANT,1020,SAMSUNG I9070,1
3,3,NOT IMPORTANT,1025,APPLE IPHONE 5C,1
4,4,NOT IMPORTANT,1027,SAMSUNG GALAXY S10 LTE,1


In [33]:
# Determine popular devices with at least 100 sales
pop_items = pd.DataFrame(dataset.MARKETING_NAME.value_counts())
pop_items2 = pop_items[pop_items.MARKETING_NAME > 100]
pop_items2 = pop_items2.reset_index()

pop_items_l = list(pop_items2.iloc[:,0])
len(pop_items_l)

974

In [34]:
#Reduce Dataset to contain only the popular items 
dataset = dataset[dataset['MARKETING_NAME'].isin(pop_items_l)]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18701718 entries, 0 to 18723356
Data columns (total 5 columns):
Unnamed: 0        int64
REFERENCE_DATE    object
SUB_ID            int64
MARKETING_NAME    object
INTERACTIONS      int64
dtypes: int64(3), object(2)
memory usage: 856.1+ MB


In [6]:
#Import Device Launch Year -- to be replaced with non-static when pipeline is implemented

bucket='bt-data-science-playground' # Or whatever you called your bucket
data_key = 'GP/device_list_all.csv' # Where the file is within your bucket
data_location = 's3://{}/{}'.format(bucket, data_key)
device_specs = pd.read_csv(data_location)
device_specs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27663 entries, 0 to 27662
Data columns (total 3 columns):
TAC               27663 non-null int64
MARKETING_NAME    27663 non-null object
YEAR_OUT          27077 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 648.4+ KB


In [7]:
# Some Devices come against multiple TACs but year is the same so we're removing duplicates 
max_device_specs = pd.DataFrame(device_specs.groupby(['MARKETING_NAME'])['YEAR_OUT'].max()).reset_index()
max_device_specs.head()

,MARKETING_NAME,YEAR_OUT
0,A-LINK TELECOM INTERNATIONAL CO LIMITED CUBOT ...,2015.0
1,ABTEL JOINT STOCK COMPANY Q SMART QS08,2014.0
2,ACER INCORPORATED E110,2010.0
3,ACER INCORPORATED E320 EU,2011.0
4,ACER INCORPORATED LIQUID JADE Z,2015.0


In [8]:
#Import Device Type - Smartphone, Tablet, Wearable -- to be replaced with non-static when pipeline is implemented

bucket='bt-data-science-playground' # Or whatever you called your bucket
data_key = 'GP/device_list_type.csv' # Where the file is within your bucket
data_location = 's3://{}/{}'.format(bucket, data_key)
device_type = pd.read_csv(data_location)
device_type.head()

,TAC,MARKETING_NAME,DEVICE_TYPE
0,35863407,APPLE IPHONE SE,Smartphone
1,35664508,APPLE IPHONE 6,Smartphone
2,35930506,APPLE IPHONE 6,Smartphone
3,35634010,APPLE IPHONE 11,Smartphone
4,35207207,APPLE IPAD AIR 2,Smartphone


In [9]:
#Same issue with TACs so we dedupe
max_product_type =  pd.DataFrame(device_type.groupby(['MARKETING_NAME'])['DEVICE_TYPE'].max()).reset_index()
max_product_type.head()

,MARKETING_NAME,DEVICE_TYPE
0,A-LINK TELECOM INTERNATIONAL CO LIMITED CUBOT ...,Smartphone
1,ABTEL JOINT STOCK COMPANY Q SMART QS08,Smartphone
2,ACER INCORPORATED LIQUID JADE Z,Smartphone
3,ACER INCORPORATED LIQUID Z4,Smartphone
4,"ALCATEL ""PIXI46""""4GANDROID""",Smartphone


In [10]:
#Import OS type

bucket='bt-data-science-playground' # Or whatever you called your bucket
data_key = 'GP/device_list_os.csv' # Where the file is within your bucket
data_location = 's3://{}/{}'.format(bucket, data_key)
device_os = pd.read_csv(data_location)
device_os['MARKETING_NAME'] = device_os['MARKETING_NAME'].map(lambda x: x.rstrip())
device_os.head()

,TAC,MARKETING_NAME,IOS
0,35299609,APPLE IPHONE 8,1
1,35233608,SAMSUNG GALAXY J7 PRIME,0
2,35285209,SONY XPERIAXA2ULTRA,0
3,35182007,SAMSUNG GALAXY A3 LTE,0
4,35195208,SAMSUNG GALAXY S7 G930F LTE,0


In [11]:
#Same issue with TACs so we dedupe
max_product_os =  pd.DataFrame(device_os.groupby(['MARKETING_NAME'])['IOS'].max()).reset_index()
max_product_os.head()

,MARKETING_NAME,IOS
0,A-LINK TELECOM INTERNATIONAL CO LIMITED CUBOT ...,0
1,ABTEL JOINT STOCK COMPANY Q SMART QS08,0
2,ACER INCORPORATED E110,0
3,ACER INCORPORATED E320 EU,0
4,ACER INCORPORATED LIQUID JADE Z,0


In [13]:
# Create Device type by code instead of Marketing Name 
max_product_os =  pd.DataFrame(dataset.groupby(['code'])['IOS'].max()).reset_index()
max_product_os.head()

,code,IOS
0,0,0.0
1,1,NaN
2,2,0.0
3,3,0.0
4,4,0.0


In [14]:
# Create Device type by code instead of Marketing Name 
max_product_type =  pd.DataFrame(dataset.groupby(['code'])['DEVICE_TYPE'].max()).reset_index()
max_product_type.head()

,code,DEVICE_TYPE
0,0,NaN
1,1,NaN
2,2,NaN
3,3,Tablet
4,4,Tablet


In [15]:
# Create Year of Launch for each user so we can take the latest device date for each user 
max_cust_year_prod =  pd.DataFrame(dataset.groupby(['code_user'])['YEAR_OUT'].max()).reset_index()
max_cust_year_prod.head()

,code_user,YEAR_OUT
0,0,2017.0
1,1,2016.0
2,2,2012.0
3,3,2013.0
4,4,2019.0


In [16]:
# Take Year of Launch by device to compare 
max_dev_year_prod =  pd.DataFrame(dataset.groupby(['code'])['YEAR_OUT'].max()).reset_index()
max_dev_year_prod.head()

,code,YEAR_OUT
0,0,2011.0
1,1,NaN
2,2,2017.0
3,3,2018.0
4,4,2018.0


In [17]:
# Take Popular Devices
pop_devices =  pd.DataFrame(dataset.groupby(['code'])['INTERACTIONS'].sum()).reset_index()
pop_devices.head()

,code,INTERACTIONS
0,0,301
1,1,3275
2,2,1991
3,3,9977
4,4,659


In [18]:
pop_devices = pop_devices.sort_values(by = 'INTERACTIONS', ascending = False)
pop_devices2 = pop_devices.head(100)
pop_devices2['Popular'] = 1 
pop_devices2.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,code,INTERACTIONS,Popular
65,65,1488364,1
63,63,1096496,1
70,70,991567,1
60,60,731323,1
67,67,710184,1


In [19]:
# Create dataset of all existing products by user in last 2 years so we can remove them later 
existing_products = dataset[['code_user','code']]
#existing_products['code'] = str(existing_products['code'])
existing_products.head()

,code_user,code
0,0,728
1,1,911
2,2,795
3,3,58
4,4,732


In [20]:
# Create Year of Launch for each user so we can take the latest device date for each user 
max_cust_os =  pd.DataFrame(dataset.groupby(['code_user'])['IOS'].max()).reset_index()
max_cust_os.head()

,code_user,IOS
0,0,0.0
1,1,0.0
2,2,0.0
3,3,1.0
4,4,0.0


In [21]:
# Scoring chunk size 
n = 3000000

In [22]:
# Chunk score dataset 
list_df = [dataset[i:i+n] for i in range(0,dataset.shape[0],n)]

In [23]:
#Number of chunks 
max_len_list = math.ceil(len(dataset)/n) 
max_len_list

7

In [28]:
def model_creation_and_scoring(num_reccs):
    #Function to explode a comma 

    def explode(df, lst_cols, fill_value='', preserve_index=False):
        # make sure `lst_cols` is list-alike
        if (lst_cols is not None
            and len(lst_cols) > 0
            and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
            lst_cols = [lst_cols]
        # all columns except `lst_cols`
        idx_cols = df.columns.difference(lst_cols)
        # calculate lengths of lists
        lens = df[lst_cols[0]].str.len()
        # preserve original index values    
        idx = np.repeat(df.index.values, lens)
        # create "exploded" DF
        res = (pd.DataFrame({
                    col:np.repeat(df[col].values, lens)
                    for col in idx_cols},
                    index=idx)
                 .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                                for col in lst_cols}))
        # append those rows that have empty lists
        if (lens == 0).any():
            # at least one list in cells is empty
            res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                      .fillna(fill_value))
        # revert the original index order
        res = res.sort_index()
        # reset index if requested
        if not preserve_index:        
            res = res.reset_index(drop=True)
        return res

    for i in range(0,max_len_list): #loops through dataset chunks to create model & score for each:
        var = os.path.exists("George/Model_Parts/recommender_"+str(i)+"/")
        if not var  :
            print('For batch #',i)
            #Transforms IDs to numeric
            list_df[i].SUB_ID = pd.to_numeric(list_df[i].SUB_ID)
            
            print('Step 1')
            print(list_df[i].head())
            
            #Transforms product to categorical
            list_df[i].MARKETING_NAME = pd.Categorical(list_df[i].MARKETING_NAME)
            
            #Lists products
            list_df[i]['code'] = list_df[i].MARKETING_NAME.cat.codes
            #dataset['code'] = dataset['code'].apply(pd.to_numeric) + 1 
            #dataset.code = dataset.code.astype(int)
            print('Step 2')
            print(list_df[i]['code'].sort_values())
            print(list_df[i].info())

            #Encodes product
            encodings_product = list_df[i][['MARKETING_NAME', 'code']]
            print('Step 3')
            print(encodings_product.info())

            encodings_product = encodings_product.drop_duplicates()

            print('Step 4')
            print(encodings_product.sort_values('code'))

            #Lists SUBs
            list_df[i].SUB_ID = pd.Categorical(list_df[i].SUB_ID)
            list_df[i]['code_user'] = list_df[i].SUB_ID.cat.codes
            #dataset['code_user'] = dataset['code_user'].apply(pd.to_numeric) + 1 
            #dataset.code_user = dataset.code_user.astype(int)
            print('Step 5')
            print(list_df[i]['code_user'].sort_values())

            #Encodes SUBs
            encodings_user = list_df[i][['SUB_ID', 'code_user']]
            print('Step 6')
            print(encodings_user.info())

            encodings_user = encodings_user.drop_duplicates()

            print('Step 7')
            print(encodings_user.sort_values('SUB_ID'))


            del list_df[i]['MARKETING_NAME']
            del list_df[i]['code_user']
            print('Step 8')
            print(list_df[i].head())

            #Arranges dataframe in correct format
            list_df[i] = list_df[i][['SUB_ID', 'code', 'INTERACTIONS', 'REFERENCE_DATE']]
            print('Step 9')
            print(list_df[i].tail(2))

            dataset2 = [list_df[i].columns.values.tolist()] + list_df[i].values.tolist()
            dataset_header = dataset2.pop(0)
            
            # Maps users & Products
            
            dataset_to_internal_user_ids = defaultdict(lambda: len(dataset_to_internal_user_ids))
            dataset_to_internal_item_ids = defaultdict(lambda: len(dataset_to_internal_item_ids))
            for row in dataset2:
                row[0] = dataset_to_internal_user_ids[int(row[0])]
                row[1] = dataset_to_internal_item_ids[int(row[1])]
                row[2] = float(row[2])
            n_users = len(dataset_to_internal_user_ids)
            n_items = len(dataset_to_internal_item_ids)

            print('Step 10')
            # Look at an example raw rating
            print("Raw ratings example:\n{}\n{}".format(dataset_header, dataset2[0]))

            # Shuffle the ratings and split them in to train/test sets 80%/20%
            random.shuffle(dataset2)  # Shuffles the list in-place
            cutoff = int(.8 * len(dataset2))
            train_ratings = dataset2[:cutoff]
            test_ratings = dataset2[cutoff:]
            print('Step 11')
            print("{} train ratings, {} test ratings".format(len(train_ratings), len(test_ratings)))


            # This method converts a list of (user, item, rating, time) to a sparse matrix
            def interactions_list_to_sparse_matrix(interactions):
                users_column, items_column, ratings_column, _ = zip(*interactions)
                return sparse.coo_matrix((ratings_column, (users_column, items_column)),
                                         shape=(n_users, n_items))

            print('Step 12')
            # Create sparse matrices of interaction data
            sparse_train_ratings = interactions_list_to_sparse_matrix(train_ratings)
            sparse_test_ratings = interactions_list_to_sparse_matrix(test_ratings)

            print('Step 13')
            # Construct indicator features for users and items
            user_indicator_features = sparse.identity(n_users)
            item_indicator_features = sparse.identity(n_items)

            

            #This method consumes item ranks for each user and prints out recall@10 train/test metrics
            def check_results(ranks):
                train_recall_at_10 = tensorrec.eval.recall_at_k(
                    test_interactions=sparse_train_ratings,
                    predicted_ranks=ranks,
                    k=10
                ).mean()
                test_recall_at_10 = tensorrec.eval.recall_at_k(
                    test_interactions=sparse_test_ratings,
                    predicted_ranks=ranks,
                    k=10
                ).mean()
                today = date.today()
                d1 = today.strftime("%d/%m/%Y")
                print("On Date: ",str(d1)," For Batch #", i , " Recall at 10: Train: {:.4f} Test: {:.4f}".format(train_recall_at_10,
                                                                        test_recall_at_10),file=open('accuracies.txt','a'))
            print('Step 14')

            from tensorflow.keras.models import Sequential
            # Let's try a new loss function: WMRB
            print("Training collaborative filter with WMRB loss")
            ranking_cf_model = tensorrec.TensorRec(n_components=5,
                                                   loss_graph=tensorrec.loss_graphs.WMRBLossGraph())
            ranking_cf_model.fit(interactions=sparse_train_ratings,
                                 user_features=user_indicator_features,
                                 item_features=item_indicator_features,
                                 n_sampled_items=int(n_items * .01),
                                 user_batch_size= 100000)

            print('Step 15')
            # Check the results of the WMRB MF CF model
            print("WMRB matrix factorization collaborative filter:")
            predicted_ranks = ranking_cf_model.predict_rank(user_features=user_indicator_features,
                                                            item_features=item_indicator_features)
            print(check_results(predicted_ranks))

            products_df = pd.DataFrame(list(dataset_to_internal_item_ids.items()), columns=['ORIG_CODE', 'SECOND_CODE'])
            products_df = pd.merge(products_df, encodings_product, how='inner', left_on='ORIG_CODE', right_on='code')

            #tensorrec.TensorRec.save_model(ranking_cf_model, "George/Model_Parts/recommender_"+str(i)+"/") -- Optional for storage 
            #print('Model Saved')
            print('Begin Scoring')
        
            #ranking_cf_model= tensorrec.TensorRec.load_model( "George/Model_Parts/")
            
            def scoring_recommender_baseline(num_reccs):
                predicted_ranks_df = pd.DataFrame(predicted_ranks)
                df1 = predicted_ranks_df.lt(num_reccs +1, 0)
                print('Picked the first ' +str(num_reccs)+ ' products for each customer')
                print(df1)
                df1 = df1.apply(lambda x: ', '.join(x.index[x].astype(str)),axis=1)
                df2 = pd.DataFrame(df1).reset_index()
                df2.columns = ['SUB_CODE','products']
                print(df2)
                df3 = explode(df2.assign(products=df2.products.str.split(',')), 'products')
                df3['products'] = df3['products'].astype(int)
                print(df3)

                products_df = pd.DataFrame(list(dataset_to_internal_item_ids.items()), columns=['ORIG_CODE', 'SECOND_CODE'])
                products_df = pd.merge(products_df, encodings_product, how='inner', left_on='ORIG_CODE', right_on='code')
                products_df = products_df.drop(columns=['SECOND_CODE', 'code'])
                print(products_df)

                df4 = pd.merge(df3, products_df, how='left', left_on='products', right_on='ORIG_CODE')
                df4 = df4.drop(columns=['products', 'ORIG_CODE'])
                df4 = pd.merge(df4, encodings_user, how='left', left_on='SUB_CODE', right_on='code_user')
                df4 = df4.drop(columns=['SUB_CODE', 'code_user'])
                df4 = df4[['SUB_ID','MARKETING_NAME']]

                df4 = pd.DataFrame(df4.groupby('SUB_ID')['MARKETING_NAME'].apply(lambda x: "{%s}" % ', '.join(x)))
                print(df4)
                df4.to_csv('George/Model_Parts/scores_'+str(i)+'/scored_set_baseline.csv')
                print('Finished scoring for Baseline Model batch #', i)
                
            def scoring_new_devices_only(num_reccs):
                #ranking_cf_model= tensorrec.TensorRec.load_model( "George/Model_Parts/Tensorrec0/")
                predicted_ranks_df = pd.DataFrame(predicted_ranks)
                df1 = predicted_ranks_df.lt(num_reccs +1, 0)
                print('Picked the first ' +str(num_reccs)+ ' products for each customer')
                print(df1)
                df1 = df1.apply(lambda x: ', '.join(x.index[x].astype(str)),axis=1)
                df2 = pd.DataFrame(df1).reset_index()
                df2.columns = ['SUB_CODE','products']
                print(df2)
                df3 = explode(df2.assign(products=df2.products.str.split(',')), 'products')
                df3['products'] = df3['products'].astype(int)
                print(df3)

                df3_3 = pd.merge(df3, max_cust_year_prod, how='left', left_on=['SUB_CODE'], right_on=['code_user'])
                df3_3.columns = ['SUB_CODE', 'products', 'code_user', 'YEAR_OUT_MAX']
                df3_3.reset_index()
                df3_3 = pd.merge(df3_3, max_dev_year_prod, how='left', left_on=['products'], right_on=['code'])
                df3_3 = df3_3.fillna(0)
                df3_3 = df3_3[df3_3['YEAR_OUT'] > df3_3['YEAR_OUT_MAX']]
                del df3_3['YEAR_OUT_MAX']
                del df3_3['YEAR_OUT']
                del df3_3['code_user']
                del df3_3['code']
                print(df3_3)

                df3_2 = pd.merge(df3_3, existing_products, how='left', left_on=['SUB_CODE', 'products'], right_on=['code_user','code'])
                df3_2 = df3_2.fillna('keep')
                df3_2 = df3_2.loc[df3_2['code_user'] == 'keep']
                df3_2 = df3_2[['SUB_CODE','products']]
                print(df3_2)

                products_df = pd.DataFrame(list(dataset_to_internal_item_ids.items()), columns=['ORIG_CODE', 'SECOND_CODE'])
                products_df = pd.merge(products_df, encodings_product, how='inner', left_on='ORIG_CODE', right_on='code')
                products_df = products_df.drop(columns=['SECOND_CODE', 'code'])
                print(products_df)

                df4 = pd.merge(df3_2, products_df, how='left', left_on='products', right_on='ORIG_CODE')
                df4 = df4.drop(columns=['products', 'ORIG_CODE'])
                df4 = pd.merge(df4, encodings_user, how='left', left_on='SUB_CODE', right_on='code_user')
                df4 = df4.drop(columns=['SUB_CODE', 'code_user'])
                df4 = df4[['SUB_ID','MARKETING_NAME']]


                df4 = pd.DataFrame(df4.groupby('SUB_ID')['MARKETING_NAME'].apply(lambda x: "{%s}" % ', '.join(x)))
                print(df4)
                df4.to_csv('George/Model_Parts/scores_'+str(i)+'/scored_set_new.csv')
                print('Finished scoring for New Devices Model batch #', i)
                
            def scoring_recommender_os_devs(num_reccs):
                #ranking_cf_model= tensorrec.TensorRec.load_model( "George/Model_Parts/Tensorrec0/")
                predicted_ranks_df = pd.DataFrame(predicted_ranks)
                df1 = predicted_ranks_df.lt(num_reccs +1, 0)
                print('Picked the first ' +str(num_reccs)+ ' products for each customer')
                print(df1)
                df1 = df1.apply(lambda x: ', '.join(x.index[x].astype(str)),axis=1)
                df2 = pd.DataFrame(df1).reset_index()
                df2.columns = ['SUB_CODE','products']
                print(df2)
                df3 = explode(df2.assign(products=df2.products.str.split(',')), 'products')
                df3['products'] = df3['products'].astype(int)
                print(df3)

                df3_3 = pd.merge(df3, max_cust_os, how='left', left_on=['SUB_CODE'], right_on=['code_user'])
                df3_3.columns = ['SUB_CODE', 'products', 'code_user', 'USER_IOS']
                df3_3.reset_index()
                df3_3 = pd.merge(df3_3, max_product_os, how='left', left_on=['products'], right_on=['code'])
                df3_3 = df3_3.fillna(0)

                df3_3.head()

                df3_4 = df3_3[df3_3['USER_IOS'] == df3_3['IOS']]
                del df3_4['IOS']
                del df3_4['USER_IOS']
                del df3_4['code_user']
                del df3_4['code']
                print(df3_4)

                df3_2 = pd.merge(df3_4, existing_products, how='left', left_on=['SUB_CODE', 'products'], right_on=['code_user','code'])
                df3_2 = df3_2.fillna('keep')
                df3_2 = df3_2.loc[df3_2['code_user'] == 'keep']
                df3_2 = df3_2[['SUB_CODE','products']]
                print(df3_2)

                products_df = pd.DataFrame(list(dataset_to_internal_item_ids.items()), columns=['ORIG_CODE', 'SECOND_CODE'])
                products_df = pd.merge(products_df, encodings_product, how='inner', left_on='ORIG_CODE', right_on='code')
                products_df = products_df.drop(columns=['SECOND_CODE', 'code'])
                print(products_df)

                df4 = pd.merge(df3_2, products_df, how='left', left_on='products', right_on='ORIG_CODE')
                df4 = df4.drop(columns=['products', 'ORIG_CODE'])
                df4 = pd.merge(df4, encodings_user, how='left', left_on='SUB_CODE', right_on='code_user')
                df4 = df4.drop(columns=['SUB_CODE', 'code_user'])
                df4 = df4[['SUB_ID','MARKETING_NAME']]


                df5 = pd.DataFrame(df4.groupby('SUB_ID')['MARKETING_NAME'].apply(lambda x: "{%s}" % ', '.join(x)))
                print(df5)
                df5.to_csv('George/Model_Parts/scores_'+str(i)+'/scored_set_os.csv')
                print('Finished scoring for Same OS Model batch #', i)
                
            def scoring_recommender_complementary_devs(num_reccs):
                #ranking_cf_model= tensorrec.TensorRec.load_model( "George/Model_Parts/Tensorrec0/")
                predicted_ranks_df = pd.DataFrame(predicted_ranks)
                df1 = predicted_ranks_df.lt(num_reccs +1, 0)
                print('Picked the first ' +str(num_reccs)+ ' products for each customer')
                print(df1)
                df1 = df1.apply(lambda x: ', '.join(x.index[x].astype(str)),axis=1)
                df2 = pd.DataFrame(df1).reset_index()
                df2.columns = ['SUB_CODE','products']
                print(df2)
                df3 = explode(df2.assign(products=df2.products.str.split(',')), 'products')
                df3['products'] = df3['products'].astype(int)
                print(df3)

                df3_3 = pd.merge(df3, max_product_type, how='left', left_on=['products'], right_on=['code'])
                df3_3.head()

                df3_4 = df3_3.loc[df3_3['DEVICE_TYPE'].isin(['Tablet','Wearable'])]


                print(df3_4.head())

                del df3_4['code']
                del df3_4['DEVICE_TYPE']


                df3_2 = pd.merge(df3_4, existing_products, how='left', left_on=['SUB_CODE', 'products'], right_on=['code_user','code'])
                df3_2 = df3_2.fillna('drop')
                df3_2 = df3_2.loc[df3_2['code_user'] != 'drop']
                df3_2 = df3_2[['SUB_CODE','products']]
                print(df3_2)

                products_df = pd.DataFrame(list(dataset_to_internal_item_ids.items()), columns=['ORIG_CODE', 'SECOND_CODE'])
                products_df = pd.merge(products_df, encodings_product, how='inner', left_on='ORIG_CODE', right_on='code')
                products_df = products_df.drop(columns=['SECOND_CODE', 'code'])
                print(products_df)

                df4 = pd.merge(df3_2, products_df, how='left', left_on='products', right_on='ORIG_CODE')
                df4 = df4.drop(columns=['products', 'ORIG_CODE'])
                df4 = pd.merge(df4, encodings_user, how='left', left_on='SUB_CODE', right_on='code_user')
                df4 = df4.drop(columns=['SUB_CODE', 'code_user'])
                df4 = df4[['SUB_ID','MARKETING_NAME']]




                df4 = pd.DataFrame(df4.groupby('SUB_ID')['MARKETING_NAME'].apply(lambda x: "{%s}" % ', '.join(x)))
                print(df4)
                df4.to_csv('George/Model_Parts/scores_'+str(i)+'/scored_set_comp.csv')
                print('Finished scoring for Complementary Model batch #', i)
                
            def scoring_recommender_baseline_v2(num_reccs):
                #ranking_cf_model= tensorrec.TensorRec.load_model( "George/Model_Parts/Tensorrec0/")
                predicted_ranks_df = pd.DataFrame(predicted_ranks)
                df1 = predicted_ranks_df.lt(num_reccs +1, 0)
                print('Picked the first ' +str(num_reccs)+ ' products for each customer')
                print(df1)
                df1 = df1.apply(lambda x: ', '.join(x.index[x].astype(str)),axis=1)
                df2 = pd.DataFrame(df1).reset_index()
                df2.columns = ['SUB_CODE','products']
                print(df2)
                df3 = explode(df2.assign(products=df2.products.str.split(',')), 'products')
                df3['products'] = df3['products'].astype(int)
                print(df3)
                df3_2 = pd.merge(df3, existing_products, how='left', left_on=['SUB_CODE', 'products'], right_on=['code_user','code'])
                df3_2 = df3_2.fillna('keep')
                df3_2 = df3_2.loc[df3_2['code_user'] == 'keep']
                df3_2 = df3_2[['SUB_CODE','products']]

                products_df = pd.DataFrame(list(dataset_to_internal_item_ids.items()), columns=['ORIG_CODE', 'SECOND_CODE'])
                products_df = pd.merge(products_df, encodings_product, how='inner', left_on='ORIG_CODE', right_on='code')
                products_df = products_df.drop(columns=['SECOND_CODE', 'code'])
                print(products_df)

                df4 = pd.merge(df3_2, products_df, how='left', left_on='products', right_on='ORIG_CODE')
                df4 = df4.drop(columns=['products', 'ORIG_CODE'])
                df4 = pd.merge(df4, encodings_user, how='left', left_on='SUB_CODE', right_on='code_user')
                df4 = df4.drop(columns=['SUB_CODE', 'code_user'])
                df4 = df4[['SUB_ID','MARKETING_NAME']]


                df4 = pd.DataFrame(df4.groupby('SUB_ID')['MARKETING_NAME'].apply(lambda x: "{%s}" % ', '.join(x)))
                print(df4)
                df4.to_csv('George/Model_Parts/scores_'+str(i)+'/scored_set_base_v2.csv')
                print('Finished scoring for Baseline v2 Model batch #', i)
                
            def scoring_recommender_popularity(num_reccs):
                #ranking_cf_model= tensorrec.TensorRec.load_model( "George/Model_Parts/Tensorrec0/")
                predicted_ranks_df = pd.DataFrame(predicted_ranks)
                df1 = predicted_ranks_df.lt(num_reccs +1, 0)
                print('Picked the first ' +str(num_reccs)+ ' products for each customer')
                print(df1)
                df1 = df1.apply(lambda x: ', '.join(x.index[x].astype(str)),axis=1)
                df2 = pd.DataFrame(df1).reset_index()
                df2.columns = ['SUB_CODE','products']
                print(df2)
                df3 = explode(df2.assign(products=df2.products.str.split(',')), 'products')
                df3['products'] = df3['products'].astype(int)
                print(df3)


                df3_3 = pd.merge(df3, pop_devices2, how='left', left_on=['products'], right_on=['code'])
                df3_3.head()

                df3_4 = df3_3.loc[df3_3['Popular'] == 1]


                print(df3_4.head())

                del df3_4['code']
                del df3_4['INTERACTIONS']
                del df3_4['Popular']

                df3_2 = pd.merge(df3_4, existing_products, how='left', left_on=['SUB_CODE', 'products'], right_on=['code_user','code'])
                df3_2 = df3_2.fillna('keep')
                df3_2 = df3_2.loc[df3_2['code_user'] == 'keep']
                df3_2 = df3_2[['SUB_CODE','products']]

                products_df = pd.DataFrame(list(dataset_to_internal_item_ids.items()), columns=['ORIG_CODE', 'SECOND_CODE'])
                products_df = pd.merge(products_df, encodings_product, how='inner', left_on='ORIG_CODE', right_on='code')
                products_df = products_df.drop(columns=['SECOND_CODE', 'code'])
                print(products_df)

                df4 = pd.merge(df3_2, products_df, how='left', left_on='products', right_on='ORIG_CODE')
                df4 = df4.drop(columns=['products', 'ORIG_CODE'])
                df4 = pd.merge(df4, encodings_user, how='left', left_on='SUB_CODE', right_on='code_user')
                df4 = df4.drop(columns=['SUB_CODE', 'code_user'])
                df4 = df4[['SUB_ID','MARKETING_NAME']]


                df4 = pd.DataFrame(df4.groupby('SUB_ID')['MARKETING_NAME'].apply(lambda x: "{%s}" % ', '.join(x)))
                print(df4)
                df4.to_csv('George/Model_Parts/scores_'+str(i)+'/scored_set_pop.csv')
                print('Finished scoring for Popularity Model batch #', i)
            
            scoring_recommender_baseline(num_reccs)
            
            
            scoring_recommender_popularity(num_reccs)
            
            
            scoring_recommender_baseline_v2(num_reccs)
            
            
            scoring_recommender_complementary_devs(num_reccs)
            
            
            scoring_new_devices_only(num_reccs)
            
            
            scoring_recommender_os_devs(num_reccs)
            


In [29]:
model_creation_and_scoring(20)

For batch # 1


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Step 1
         Unnamed: 0 REFERENCE_DATE      SUB_ID  \
3000000     3013057  NOT IMPORTANT  1063766267   
3000001     3013058  NOT IMPORTANT  1063766282   
3000002     3013059  NOT IMPORTANT  1063766282   
3000003     3013060  NOT IMPORTANT  1063766287   
3000004     3013061  NOT IMPORTANT  1063766289   

                           MARKETING_NAME  INTERACTIONS  code  code_user  \
3000000                SONY XPERIA Z5 LTE             1   928    2054076   
3000001                    APPLE IPHONE X             1    71    2054077   
3000002  SAMSUNG GALAXY S6 EDGE G925F LTE             1   750    2054077   
3000003                    APPLE IPHONE 5             1    57    2054078   
3000004                    APPLE IPHONE 6             1    60    2054079   

         YEAR_OUT DEVICE_TYPE  IOS  
3000000    2015.0  Smartphone  0.0  
3000001    2017.0  Smartphone  1.0  
3000002    2015.0  Smartphone  0.0  
3000003    2013.0  Smartphone  1.0  
3000004    2014.0  Smartphone  1.0  


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Step 2
5563116      0
5461839      0
4785845      1
5204688      1
5315396      1
5180175      1
4257527      1
5832469      1
4074643      2
3060981      2
5425313      2
3077178      2
3089534      2
5863744      2
4650850      2
5846748      2
4593189      2
3100629      2
3174998      2
4646911      2
4322142      2
4199671      2
3818642      2
4300477      2
4528781      2
4338684      2
5937191      2
3340201      2
4441757      2
3091078      2
          ... 
3175373    797
3386222    797
3014143    797
3004340    797
3058903    797
3032464    797
3104549    797
3139006    797
3139049    797
3139055    797
3116774    797
3104548    797
3116779    797
3139056    797
3139057    797
3012572    797
3139015    797
3079767    797
3091523    797
3101085    797
3138999    797
3139072    797
3120046    797
3082045    797
3120038    797
3139086    797
3229209    797
3042533    797
3116781    797
3017728    797
Name: code, Length: 3000000, dtype: int16
<class 'pandas.core.frame.DataFrame'

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Step 5
3000000          0
3000001          1
3000002          1
3000003          2
3000004          3
3000005          3
3000006          4
3000007          4
3000008          5
3000009          5
3000011          6
3000010          6
3000012          7
3000013          8
3000014          8
3000015          9
3000016         10
3000017         11
3000018         12
3000019         13
3000020         14
3000021         15
3000022         16
3000023         17
3000024         17
3000025         18
3000026         19
3000027         19
3000028         20
3000029         20
            ...   
5999970    2006072
5999971    2006073
5999972    2006073
5999973    2006074
5999975    2006075
5999974    2006075
5999976    2006076
5999977    2006077
5999978    2006078
5999979    2006078
5999980    2006079
5999981    2006079
5999982    2006080
5999983    2006081
5999984    2006082
5999985    2006082
5999986    2006083
5999987    2006083
5999989    2006084
5999988    2006084
5999990    2006085
59999

KeyboardInterrupt: 

In [3]:
#Import Digital Price book  -- to be replaced with non-static when pipeline is implemented
bucket='bt-data-science-playground' # Or whatever you called your bucket
data_key = 'GP/digital_pricebook.csv' # Where the file is within your bucket
data_location = 's3://{}/{}'.format(bucket, data_key)
digital_pricebook = pd.read_csv(data_location)
digital_pricebook = digital_pricebook[pd.notnull(digital_pricebook['DEVICE_NAME'])]
digital_pricebook['DEVICE_NAME'] = digital_pricebook['DEVICE_NAME'].str.upper()

digital_pricebook = digital_pricebook.fillna(0)

digital_pricebook = digital_pricebook[digital_pricebook['JOURNEY_TYPE'] == 'Upgrade']
digital_pricebook['key'] = 0 

digital_pricebook.head()

,TARIFF_CODE,DATA,MINUTES,SMS,UPFRONT_COST,MONTHLY_COST,JOURNEY_TYPE,MP_IDENTIFIER,DEVICE_NAME
7,X19B24H41,100000,Unlimited,Unlimited,0.0,104.0,Upgrade,iphone-6s-128gb-silver,IPHONE 6S 128GB SILVER
8,X19B24H05,10000,Unlimited,Unlimited,0.0,74.0,Upgrade,iphone-6s-plus-128gb-silver,IPHONE 6S PLUS 128GB SILVER
9,X19A24H57,30000,Unlimited,Unlimited,0.0,74.0,Upgrade,iphone-6s-128gb-rose-gold,IPHONE 6S 128GB ROSE GOLD
10,X19B24H22,60000,Unlimited,Unlimited,0.0,59.0,Upgrade,iphone-7-32gb-black,IPHONE 7 32GB BLACK
11,X19A24H63,60000,Unlimited,Unlimited,300.0,41.0,Upgrade,iphone-7-128gb-silver,IPHONE 7 128GB SILVER


In [ ]:
#Import Customer Predicted Usage -- to be replaced with non-static when pipeline is implemented
bucket='bt-data-science-playground' # Or whatever you called your bucket
data_key = 'GP/cust_pred_usage.csv' # Where the file is within your bucket
data_location = 's3://{}/{}'.format(bucket, data_key)
cust_pred_usage = pd.read_csv(data_location)
cust_pred_usage = cust_pred_usage[['SUB_ID','PRED_USAGE']]
cust_pred_usage['SUB_ID'] = np.int64(cust_pred_usage['SUB_ID'])
cust_pred_usage.head()

In [68]:
def final_scoring_w_tariff(table_name):
    for i in range(0,6): #loops through dataset chunks to create model & score for each:
        location = 'George/Model_Parts/scores_' + str(i) + '/' + table_name +'.csv'
        d_score = pd.read_csv(location)
        d_score['MARKETING_NAME'] = d_score['MARKETING_NAME'].str.replace('{','')
        d_score['MARKETING_NAME'] = d_score['MARKETING_NAME'].str.replace('}','')
        d_score['MARKETING_NAME'] = d_score['MARKETING_NAME'].str.replace('LTE','')
        print(d_score.head())

        d_score =  pd.DataFrame(d_score.MARKETING_NAME.str.split(',').tolist(), index=d_score.SUB_ID).stack()
        print(d_score.head())

        d_score = d_score.reset_index([0, 'SUB_ID'])
        d_score.columns = ['SUB_ID', 'MARKETING_NAME']
        print(d_score.head())

        recc_devices = pd.DataFrame(d_score['MARKETING_NAME'].unique())
        recc_devices.columns = ['MARKETING_NAME']
        print(recc_devices.nunique())

        recc_devices['key'] = 0 

        recc_devices2 = recc_devices.merge(digital_pricebook, how='outer' , on ='key')
        recc_devices2 = recc_devices2.dropna()
        print(recc_devices2.head())

        recc_devices2['MARKETING_NAME'] = recc_devices2['MARKETING_NAME'].astype(str)
        recc_devices2['MARKETING_NAME'] = recc_devices2['MARKETING_NAME'].map(lambda x: x.rstrip())
        recc_devices2['DEVICE_NAME'] = recc_devices2['DEVICE_NAME'].astype(str)
        recc_devices2['DEVICE_NAME'] = recc_devices2['DEVICE_NAME'].map(lambda x: x.rstrip())
        print(recc_devices2.info())

        recc_devices2['same'] = [x[0] in x[1] for x in zip(recc_devices2['MARKETING_NAME'], recc_devices2['DEVICE_NAME'])]
        print(recc_devices2.head())

        recc_devices3 = recc_devices2

        #recc_devices3['partial_ratio'] = recc_devices3.apply(lambda x: fuzz.partial_ratio(x['MARKETING_NAME'], x['DEVICE_NAME']), axis=1)
        #recc_devices3['ratio'] = recc_devices3.apply(lambda x: fuzz.ratio(x['MARKETING_NAME'], x['DEVICE_NAME']), axis=1)
        recc_devices3['token_sort_ratio'] = recc_devices3.apply(lambda x: fuzz.token_sort_ratio(x['MARKETING_NAME'], x['DEVICE_NAME']), axis=1)
        #recc_devices3['token_set_ratio'] = recc_devices3.apply(lambda x: fuzz.token_set_ratio(x['MARKETING_NAME'], x['DEVICE_NAME']), axis=1)
        #recc_devices3['avg_ratio'] = ( recc_devices3['partial_ratio'] + recc_devices3['ratio'] + recc_devices3['token_sort_ratio'] + recc_devices3['token_set_ratio']) / 4 

        print(recc_devices3.head())

        recc_devices4 = recc_devices3[recc_devices3['token_sort_ratio'] > 70]
        recc_devices4['DATA'].replace('Unlimited','99999999',inplace=True)
        recc_devices4['DATA'] = pd.to_numeric(recc_devices4['DATA'])
        print(recc_devices4.head())

        d_score = d_score.merge(cust_pred_usage, how='inner' , on ='SUB_ID')
        print(d_score.head())

        # Scoring chunk size 
        n = 100000

        # Chunk score dataset 
        list_df = [d_score[i:i+n] for i in range(0,d_score.shape[0],n)]

        #Number of chunks 
        max_len_list2 = math.ceil(len(d_score)/n) 
        print(max_len_list2)

        scored_base = pd.DataFrame()

        for j in range(0,max_len_list2): #loops through dataset chunks to create model & score for each:
            if table_name == 'scored_set_comp':
                list_df[j]['MARKETING_NAME'] = list_df[j]['MARKETING_NAME'].astype(str)
                list_df[j]['MARKETING_NAME'] = list_df[j]['MARKETING_NAME'].map(lambda x: x.rstrip())
                list_df[j] = list_df[j].merge(recc_devices4, how='left' , on ='MARKETING_NAME')
                #list_df[j] = list_df[j][list_df[j]['DATA'] > list_df[j]['PRED_USAGE']]
                idx = list_df[j].groupby(['SUB_ID','MARKETING_NAME'])['DATA'].transform(min) == list_df[j]['DATA']
                list_df[j] = list_df[j][idx]
                list_df[j] = list_df[j].groupby(['SUB_ID','MARKETING_NAME']).apply(lambda x: x.sample(1)).reset_index(drop=True)
                scored_base = scored_base.append(list_df[j])
                print(list_df[j].head())
            else :
                list_df[j]['MARKETING_NAME'] = list_df[j]['MARKETING_NAME'].astype(str)
                list_df[j]['MARKETING_NAME'] = list_df[j]['MARKETING_NAME'].map(lambda x: x.rstrip())
                list_df[j] = list_df[j].merge(recc_devices4, how='left' , on ='MARKETING_NAME')
                list_df[j] = list_df[j][list_df[j]['DATA'] > list_df[j]['PRED_USAGE']]
                idx = list_df[j].groupby(['SUB_ID','MARKETING_NAME'])['DATA'].transform(min) == list_df[j]['DATA']
                list_df[j] = list_df[j][idx]
                list_df[j] = list_df[j].groupby(['SUB_ID','MARKETING_NAME']).apply(lambda x: x.sample(1)).reset_index(drop=True)
                scored_base = scored_base.append(list_df[j])
                print(list_df[j].head())
        scored_base.to_csv('George/Model_Parts/scores_'+str(i)+'/' + table_name +'_final.csv')


In [69]:
final_scoring_w_tariff('scored_set_comp')
final_scoring_w_tariff('scored_set_base_v2')
final_scoring_w_tariff('scored_set_baseline')
final_scoring_w_tariff('scored_set_new')
final_scoring_w_tariff('scored_set_os')
final_scoring_w_tariff('scored_set_pop')

   SUB_ID MARKETING_NAME
0    1006               
1    1016               
2    1020               
3    1025               
4    1027               
SUB_ID   
1006    0    
1016    0    
1020    0    
1025    0    
1027    0    
dtype: object
   SUB_ID MARKETING_NAME
0    1006               
1    1016               
2    1020               
3    1025               
4    1027               
MARKETING_NAME    27
dtype: int64
  MARKETING_NAME  key TARIFF_CODE    DATA    MINUTES        SMS  UPFRONT_COST  \
0                   0   X19B24H41  100000  Unlimited  Unlimited           0.0   
1                   0   X19B24H05   10000  Unlimited  Unlimited           0.0   
2                   0   X19A24H57   30000  Unlimited  Unlimited           0.0   
3                   0   X19B24H22   60000  Unlimited  Unlimited           0.0   
4                   0   X19A24H63   60000  Unlimited  Unlimited         300.0   

   MONTHLY_COST JOURNEY_TYPE                MP_IDENTIFIER  \
0         104.0      Upg

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


   SUB_ID MARKETING_NAME  PRED_USAGE
0    1006                     3300.0
1    1016                    11900.0
2    1020                    10100.0
3    1025                     1400.0
4    1027                      500.0
15


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
       SUB_ID         MARKETING_NAME  PRED_USAGE  key TARIFF_CODE   DATA  \
0  1037363957  SAMSUNG GALAXY TAB S2     19300.0  0.0   X19A24H07  500.0   
1  1037950021  SAMSUNG GALAXY TAB S2     11200.0  0.0   X19A24H08  500.0   

     MINUTES        SMS  UPFRONT_COST  MONTHLY_COST JOURNEY_TYPE  \
0  Unlimited  Unlimited          30.0          38.0      Upgrade   
1  Unlimited  Unlimited         100.0          43.0      Upgrade   

        MP_IDENTIFIER         DEVICE_NAME   same  token_sort_ratio  
0   samsung-galaxy-a9   SAMSUNG GALAXY A9  False              74.0  
1  samsung-galaxy-a80  SAMSUNG GALAXY A80  False              72.0  
       SUB_ID    

FileNotFoundError: [Errno 2] File b'George/Model_Parts/scores_1/scored_set_comp.csv' does not exist: b'George/Model_Parts/scores_1/scored_set_comp.csv'